In [11]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
import threading
import time

class AI_Gym_Trainer:
    def __init__(self):
        # 1. Setup MediaPipe Pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_pose = mp.solutions.pose
        
        # 2. Setup Video Capture (High Res)
        self.cap = cv2.VideoCapture(0)
        self.width = 1280
        self.height = 720
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.height)
        
        # 3. Exercise State
        self.counter = 0 
        self.stage = "down" # down or up
        self.current_exercise = "bicep_curl" # Options: 'bicep_curl', 'squat', 'lateral_raise'
        self.feedback_text = "Get Ready"
        self.instruction_text = "Curl your arm up" # Step-by-step guide
        self.form_good = True
        
        # 4. Audio Setup (Threaded)
        self.engine = pyttsx3.init()
        self.engine.setProperty('rate', 160)
        self.last_speech_time = 0
        self.speech_lock = threading.Lock()

    def speak(self, text):
        """Threaded text-to-speech to prevent video lag."""
        now = time.time()
        # 3-second cooldown to avoid spamming
        if now - self.last_speech_time > 3.0:
            self.last_speech_time = now
            def _speak_thread():
                try:
                    with self.speech_lock:
                        self.engine.say(text)
                        self.engine.runAndWait()
                except: pass
            threading.Thread(target=_speak_thread, daemon=True).start()

    def calculate_angle(self, a, b, c):
        """Calculates angle between three points (a-b-c)."""
        a = np.array(a) # First
        b = np.array(b) # Mid
        c = np.array(c) # End
        
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)
        
        if angle > 180.0:
            angle = 360-angle
        return angle

    def draw_dashboard(self, image, progress_0_to_1):
        """Draws a clean, professional dashboard on the left side."""
        # 1. Sidebar Background
        cv2.rectangle(image, (0, 0), (350, 720), (255, 255, 255), -1) # White background
        cv2.rectangle(image, (0, 0), (350, 720), (200, 200, 200), 2)  # Border
        
        # 2. Exercise Title
        title = self.current_exercise.replace("_", " ").upper()
        cv2.putText(image, title, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (50, 50, 50), 2)
        
        # 3. Rep Counter Box
        cv2.rectangle(image, (20, 80), (330, 200), (245, 117, 16), -1) # Blueish box
        cv2.putText(image, str(self.counter), (120, 170), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 5)
        cv2.putText(image, "REPS", (135, 190), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        # 4. Progress Bar (Visualizing the movement range)
        bar_x, bar_y, bar_w, bar_h = 20, 230, 310, 30
        cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_w, bar_y + bar_h), (200, 200, 200), -1)
        fill_w = int(bar_w * progress_0_to_1)
        # Color changes based on form
        bar_color = (0, 255, 0) if self.form_good else (0, 0, 255)
        cv2.rectangle(image, (bar_x, bar_y), (bar_x + fill_w, bar_y + bar_h), bar_color, -1)
        
        # 5. Instructions & Feedback Area
        cv2.putText(image, "INSTRUCTION:", (20, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (50, 50, 50), 1)
        # Wrap text logic roughly
        words = self.instruction_text.split(' ')
        line1 = " ".join(words[:4])
        line2 = " ".join(words[4:])
        cv2.putText(image, line1, (20, 335), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
        cv2.putText(image, line2, (20, 365), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)

        # 6. Correction Feedback (Dynamic)
        if not self.form_good:
            cv2.rectangle(image, (20, 400), (330, 550), (0, 0, 255), 2) # Red alert box
            cv2.putText(image, "CORRECTION:", (30, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
            # Simple word wrap for feedback
            f_words = self.feedback_text.split(' ')
            for i in range(0, len(f_words), 3):
                line = " ".join(f_words[i:i+3])
                cv2.putText(image, line, (30, 465 + i*15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        else:
            cv2.putText(image, "PERFECT FORM", (40, 460), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)

        # 7. Menu Keys
        cv2.putText(image, "Press '1': Curls", (20, 620), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 100), 1)
        cv2.putText(image, "Press '2': Squats", (20, 645), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 100), 1)
        cv2.putText(image, "Press '3': Lateral Raise", (20, 670), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 100), 1)
        cv2.putText(image, "Press 'q': Quit", (20, 695), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 100), 1)

    def process_bicep_curl(self, landmarks):
        # 1. Get Coordinates
        shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].y]
        hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
               landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]

        # 2. Calculate Angles
        angle = self.calculate_angle(shoulder, elbow, wrist)
        # Check for swinging (upper arm vs torso)
        swing_angle = self.calculate_angle(hip, shoulder, elbow)

        # 3. Analyze Form
        self.form_good = True
        
        # Check: Swinging Elbow?
        if swing_angle > 30: # 30 degrees tolerance
            self.form_good = False
            self.feedback_text = "Elbow is swinging! Keep it tucked."
            self.speak("Keep your elbow tucked to your side")
        
        # 4. Count Reps & Set Instructions
        progress = 0
        if angle > 160:
            self.stage = "down"
            self.instruction_text = "Squeeze biceps and curl UP."
            progress = 0
        if angle < 30 and self.stage == "down":
            self.stage = "up"
            self.counter += 1
            self.speak(str(self.counter))
            self.instruction_text = "Lower slowly to start."
            progress = 1
        
        # Map angle to 0-1 for progress bar (approximate)
        if self.stage == "down":
            progress = max(0, min(1, (160 - angle) / 130))
        else:
            progress = max(0, min(1, (angle - 30) / 130))

        return progress

    def process_squat(self, landmarks):
        hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
               landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].y]
        shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                   landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]

        angle = self.calculate_angle(hip, knee, ankle)
        back_angle = self.calculate_angle(shoulder, hip, knee)
        
        self.form_good = True
        
        # Check: Leaning forward?
        if back_angle < 50:
            self.form_good = False
            self.feedback_text = "Too much leaning! Keep chest UP."
            self.speak("Chest up")

        # Rep Logic
        if angle > 165:
            self.stage = "up"
            self.instruction_text = "Lower your hips down."
        if angle < 90 and self.stage == "up":
            self.stage = "down"
            self.counter += 1
            self.speak(str(self.counter))
            self.instruction_text = "Drive up through heels."
            
        # Check: Not Deep Enough?
        if self.stage == "up" and angle < 140 and angle > 100:
             self.feedback_text = "Go Deeper!"
        
        # Progress calculation (Standing=170, Squat=80)
        norm_angle = np.interp(angle, [80, 170], [1, 0])
        return norm_angle

    def process_lateral_raise(self, landmarks):
        shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].y]
        hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
               landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]

        # Angle of arm lifting up from side
        lift_angle = self.calculate_angle(hip, shoulder, elbow)
        # Angle of straight arm
        straight_arm_angle = self.calculate_angle(shoulder, elbow, wrist)

        self.form_good = True
        
        # Check: Bent Arm?
        if straight_arm_angle < 140:
            self.form_good = False
            self.feedback_text = "Straighten your arm!"
            self.speak("Straighten arm")

        # Rep Logic
        if lift_angle < 25:
            self.stage = "down"
            self.instruction_text = "Lift arm to side."
        if lift_angle > 80 and self.stage == "down":
            self.stage = "up"
            self.counter += 1
            self.speak(str(self.counter))
            self.instruction_text = "Lower arm slowly."

        # Progress (0=side, 1=shoulder height)
        norm_angle = np.interp(lift_angle, [20, 90], [0, 1])
        return norm_angle

    def run(self):
        # High confidence for detection/tracking to focus on the main person
        with self.mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
            while self.cap.isOpened():
                ret, frame = self.cap.read()
                if not ret: break

                # 1. Preprocessing
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                progress = 0
                
                # 2. Pose Processing
                if results.pose_landmarks:
                    # Draw Skeleton
                    self.mp_drawing.draw_landmarks(
                        image, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS,
                        self.mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                        self.mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                    )
                    
                    landmarks = results.pose_landmarks.landmark
                    
                    try:
                        if self.current_exercise == 'bicep_curl':
                            progress = self.process_bicep_curl(landmarks)
                        elif self.current_exercise == 'squat':
                            progress = self.process_squat(landmarks)
                        elif self.current_exercise == 'lateral_raise':
                            progress = self.process_lateral_raise(landmarks)
                    except:
                        pass
                else:
                    self.instruction_text = "Please step into frame"
                    self.form_good = True

                # 3. Draw The Dashboard
                self.draw_dashboard(image, progress)

                # 4. Display
                cv2.imshow('AI Gym Trainer Pro', image)

                # 5. Inputs
                key = cv2.waitKey(10) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('1'):
                    self.current_exercise = 'bicep_curl'
                    self.counter = 0
                elif key == ord('2'):
                    self.current_exercise = 'squat'
                    self.counter = 0
                elif key == ord('3'):
                    self.current_exercise = 'lateral_raise'
                    self.counter = 0

            self.cap.release()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    trainer = AI_Gym_Trainer()
    trainer.run()

In [10]:

import cv2
import mediapipe as mp
import numpy as np
import PoseModule as pm



cap = cv2.VideoCapture(0)
detector = pm.poseDetector()
count = 0
direction = 0
form = 0
feedback = "Fix Form"


while cap.isOpened():
    ret, img = cap.read() #640 x 480
    #Determine dimensions of video - Help with creation of box in Line 43
    width  = cap.get(3)  # float `width`
    height = cap.get(4)  # float `height`
    # print(width, height)
    
    img = detector.findPose(img, False)
    lmList = detector.findPosition(img, False)
    # print(lmList)
    if len(lmList) != 0:
        elbow = detector.findAngle(img, 11, 13, 15)
        shoulder = detector.findAngle(img, 13, 11, 23)
        hip = detector.findAngle(img, 11, 23,25)
        
        #Percentage of success of pushup
        per = np.interp(elbow, (90, 160), (0, 100))
        
        #Bar to show Pushup progress
        bar = np.interp(elbow, (90, 160), (380, 50))

        #Check to ensure right form before starting the program
        if elbow > 160 and shoulder > 40 and hip > 160:
            form = 1
    
        #Check for full range of motion for the pushup
        if form == 1:
            if per == 0:
                if elbow <= 90 and hip > 160:
                    feedback = "Up"
                    if direction == 0:
                        count += 0.5
                        direction = 1
                else:
                    feedback = "Fix Form"
                    
            if per == 100:
                if elbow > 160 and shoulder > 40 and hip > 160:
                    feedback = "Down"
                    if direction == 1:
                        count += 0.5
                        direction = 0
                else:
                    feedback = "Fix Form"
                        # form = 0
                
                    
    
        print(count)
        
        #Draw Bar
        if form == 1:
            cv2.rectangle(img, (580, 50), (600, 380), (0, 255, 0), 3)
            cv2.rectangle(img, (580, int(bar)), (600, 380), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, f'{int(per)}%', (565, 430), cv2.FONT_HERSHEY_PLAIN, 2,
                        (255, 0, 0), 2)


        #Pushup counter
        cv2.rectangle(img, (0, 380), (100, 480), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, str(int(count)), (25, 455), cv2.FONT_HERSHEY_PLAIN, 5,
                    (255, 0, 0), 5)
        
        #Feedback 
        cv2.rectangle(img, (500, 0), (640, 40), (255, 255, 255), cv2.FILLED)
        cv2.putText(img, feedback, (500, 40 ), cv2.FONT_HERSHEY_PLAIN, 2,
                    (0, 255, 0), 2)

        
    cv2.imshow('Pushup counter', img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.